# **Setup**
 
Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries needed to complete the code Professor Melnikov presented in the video.

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"    # allows multiple outputs from a cell
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, nltk
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
pd.set_option('max_colwidth', 100, 'display.max_rows', 10)

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Review**

## **Improve a Baseline Model**

<span style="color:black">You will improve the prior baseline model for classifying names into male and female categories ($Y=0$ and $Y=1$, respectively). To do so, you will try different features and models in hopes that some of these will result in an improved accuracy score. Essentially, you are searching for the model hyperparameters and parameters that best represent the relationship between inputs and outputs. 
    
<h2><span style="color:black"> Baseline Model: v0 </span></h2>
    
<span style="color:black"> You will build a baseline model (version 0), a logistic regression model that accepts the number of characters in the name as the input feature. Begin by loading, balancing, and processing the dataset in a Pandas DataFrame, `df0`.

In [ ]:
_ = nltk.download(['names'], quiet=True)
LsM = nltk.corpus.names.words('male.txt')
LsF = nltk.corpus.names.words('female.txt')

# Balance observations in two classes. So, a random draw has 50% chance of either class.
np.random.seed(0)
LsF = sorted(list(np.random.choice(LsF, size=len(LsM), replace=False))) 
df0 = pd.DataFrame(dict(Name=LsF + LsM, Y=[1]*len(LsF) + [0]*len(LsM)) ) # add 1 for each female and 0 for each male
df0['Len'] = df0.Name.apply(len)
df0 = df0.set_index('Name')
df0.T

<span style="color:black">Split `df0` into training and validation sets and fit a logistic regression model on the training data. When splitting data, make sure to use the same random number generator (RNG) seed (`random_state=0`) to ensure that all models are trained on the same training sample regardless of the number of features. This allows you to relate any increases or drops in performance to your model choices.

In [ ]:
tX, vX, tY, vY = train_test_split(df0.drop('Y', axis=1), df0.Y, test_size=0.2, random_state=0)
lr = LogisticRegression().fit(tX, tY)  # create a model object and train it
print(f'Accuracy = fraction of correct predictions: {lr.score(vX, vY):.3f}')

<span style="color:black"> Evaluation of the logistic regression model on the validation set yields an accuracy of 0.503. This accuracy will be the baseline accuracy with which you will use to compare your improved models. It is hoped that future models will beat the results of prior models. You will now modify features and the model to build an improved model.

## Model v1: Encode the First Character


<span style="color:black">One way to modify the model is to use a different feature as its input (i.e., changing the hypothesis). In model v1, the hypothesis is that the first character of the name is indicative of gender. To create the dataset, you will need to convert the first character of the name into a number that will be the model's input. The actual value of the number does not matter, because you want to learn associations between first letters and gender. 
    
<span style="color:black">Begin by using the [`ord()`](https://docs.python.org/3/library/functions.html#ord) function to draw a predefined numeric character code.

In [ ]:
sChars = '!.,019ABYZ[]_abyz'
LTsnChars = [ord(c) for c in sChars]
pd.set_option('max_colwidth', 100, 'display.max_columns', 100, 'display.max_rows', 8)
pd.DataFrame(LTsnChars, index=list(sChars), columns=['character code']).T

<span style="color:black"> Now, you can map the 0-th character of every name to a numeric character code and store this in `L0` feature column of the DataFrame.

In [ ]:
df1 = df0.copy()
df1['L0'] = [ord(s[0]) for s in df1.index]
df1.T

<span style="color:black">Similarly to before, split the new dataframe `df1`(with the same RNG seed of 0), fit the model on the training set, and evaluate the model on the validation set.

In [ ]:
tX, vX, tY, vY = train_test_split(df1.drop('Y', axis=1), df1.Y, test_size=0.2, random_state=0)
print(f'Accuracy: {lr.fit(tX, tY).score(vX, vY):.3f}')

<span style="color:black">The resulting accuracy is a bit better than model v0. You will include this feature in future models.

## Model v2: Encode 2nd Character

<span style="color:black">For this next model, you will try using the numerical encoding of the second character of the name as the model input.

In [ ]:
df2 = df1.copy()
df2['L1'] = [ord(s[1]) for s in df2.index]
df2.T

<span style="color:black">As before, split the dataset, train the model, and then evaluate the model.

In [ ]:
tX, vX, tY, vY = train_test_split(df2.drop('Y', axis=1), df2.Y, test_size=0.2, random_state=0)
print(f'Accuracy: {lr.fit(tX, tY).score(vX, vY):.3f}')

<span style="color:black"> This model has a lower accuracy than model v1, and so using this feature is not an improvement. You will disregard this feature in future models.

## Model v3: Encode Last Character

<span style="color:black">In model v3, you will try using an encoding of the last character of the name. It is hoped that this feature can distinguish names better (e.g., you can observe that names ending with `'o'` (like Marko, Darko, ...) are mostly male names). Add the encodings to the Dataframe as `R0`.

In [ ]:
df3 = df1.copy()
df3['R0'] = [ord(s[-1]) for s in df3.index]
df3.T

<span style="color:black">As before, split the dataset, train the model, and then evaluate the model.

In [ ]:
tX, vX, tY, vY = train_test_split(df3.drop('Y', axis=1), df3.Y, test_size=0.2, random_state=0)
print(f'Accuracy: {lr.fit(tX, tY).score(vX, vY):.3f}')

<span style="color:black">Accuracy jumped from 52% to 68%. You will include this feature in future models.

## Model v4: Use Naive Bayes Classifier

<span style="color:black">Although you can continue trying different input features to improve the model, you can also try to use a different family of classifiers. In the above four models, [logistic regression](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression) finds a linear decision boundary (or a separating hyperplane in the vector space of our features). This classifier can be fairly rigid, and there are more flexible classifiers that you can try, such as the [naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) classifier. Naive Bayes assumes all features are independent and computes the likelihood of each gender as the product of the observed proportions of each label. 
    
<span style="color:black">You will train and evaluate a default AdaBoost model with the three features: name length, first character embedding, and last character embedding.

In [ ]:
from sklearn.naive_bayes import GaussianNB
print(f'Accuracy: {GaussianNB().fit(tX, tY).score(vX, vY):.3f}')

<span style="color:black">Unfortunately, expectations did not materialize, and the accuracy dropped a bit. However, this does not invalidate this model, because you could try changing its default hyperparameters to other values. For now though, you will skip hyperparameter tuning and continue trying other models.

## Model v5: Use Random Forest Classifier

<span style="color:black">Random forest is an ensemble of decision tree models. The latter are called **strong classifiers or estimators** because they are capable of fitting nearly all training observations. Hence, strong classifiers are likely to **overfit** the training data, i.e. learn noisy and irrelevant details about the relation in the inputs $x_i$ and the corresponding outputs $y_i$. Decision tree model tends to generalize poorly, i.e. its performance suffers on unseen observations (i.e. samples not used in training of the model).

Magically, if you combine many diverse strong classifiers into a group, or an **ensemble**, which averages (or aggregates in some other way) the very noisy predictions, the noise tends to partially cancel out and the ensemble model performs "better", i.e. with lower variability in the averaged predictions and consistently closer predictions to the target (the so-called **low bias**).
    
<span style="color:black">You will train and evaluate a default Random Forest model with the three features.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
print(f'Accuracy: {RandomForestClassifier().fit(tX, tY).score(vX, vY):.3f}')

<span style="color:black">This random forest model achieved 75% accuracy, which is the best score so far. You would want to keep this model and features.

## Model v6: AdaBoost Classifier

<span style="color:black">In [AdaBoost](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html), instead of applying all decision trees simultaneously, you train one simple tree (called weak classifier) and then train the next tree on the error residuals of the previous tree and so on.
    
<span style="color:black">You will train and evaluate a default AdaBoost model with the three features.

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
print(f'Accuracy: {AdaBoostClassifier().fit(tX, tY).score(vX, vY):.3f}')

<span style="color:black">The performance jumps another half a percent, which is great!

## Model v7: Use AdaBoost Classifier With More Estimators

<span style="color:black">To improve the AdaBoost performance, you can modify the hyperparameters. By [default](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html), scikit-learn's AdaBoost model uses 50 sequential decision trees. You can increase it to 200 in hopes that the model can learn better associations, but still avoid overfitting.

In [ ]:
ab = AdaBoostClassifier(n_estimators=200)
ab.fit(tX, tY)
print(f'Accuracy: {ab.score(vX, vY):.3f}')

<span style="color:black">The accuracy improved by another 0.6%. Great!

## Make a Prediction With the Best Model

<span style="color:black">Although you can keep making improvements to the features and model, you will now use the best model to make a predicion for a name that the model has not seen before. You will use the name `'Asha'`, which has 4 letters, starts with character 65, and ends with a character 97. These three values are packaged as a vector and passed to the `predict_proba()` method. The result can be thresholded manually at 50% or you can use `predict()` to deduce the same conclusion.

In [ ]:
sName = 'Asha'
x = np.array([[len(sName), ord(sName[0]), ord(sName[-1])]])
print(f"name:\t\t\t{sName} \nobservation:\t\t{x} \nclass probabilities:\
\t{ab.predict_proba(x).round(4)} \nclass:\t\t\t{['male', 'female'][ab.predict(x)[0]]}")

<span style="color:black">The class female is predicted with 0.504 probability.

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Optional Practice**

Now, equipped with these concepts and tools, you will tackle a few related tasks.

As you work through these tasks, check your answers by running your code in the *#check solution here* cell, to see if you’ve gotten the correct result. If you get stuck on a task, click the See **solution** drop-down to view the answer.

Below is a list of names for which you will make a gender prediction. Feel free to add/remove names as you see fit.

In [ ]:
LsNames = 'Aiguo Tushar Oleg Al Chao Joe Olga Asha Kristina Katherine Katerina Svetlana'.split()
print(LsNames)

## Task 1

For each name in `LsNames` compute a vector of features, $x$, then predict probability of each class, i.e. $\mathbb{P}[Y=0|x]$ and $\mathbb{P}[Y=1|x]$, i.e. probability of a male gender given the extracted features and the probability of a female given the extracted features. Then predict the gender code $Y\in\{0,1\}$ and gender label (male or female).

<span style="color:black"> Present these results in a DataFrame.
    
<b>Hint:</b> You can create several helper functions or package all computations into a single loop. All outputs can be more presentable as a dataframe.

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
MakeVec = lambda sName: np.array([[len(sName), ord(sName[0]), ord(sName[-1])]])
Predict = lambda x, model: (*model.predict_proba(x).round(3).flatten(), model.predict(x)[0], ['male','female'][model.predict(x)[0]])
PredictInfo = lambda sName, model: (sName, MakeVec(sName).flatten(), *Predict(MakeVec(sName), ab))

pd.DataFrame([PredictInfo(sName, ab) for sName in LsNames], 
             columns=['Name','feature vector, x','P[Y=0|x]','P[Y=1|x]','predicted Y','Gender label']).set_index('Name').T
</pre>
</details> 
</font>
<hr>

## Task 2

Check the documentation for [AdaBoostClassifier ](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html) and investigate its hyperparameters such as `n_estimators`, `learning_rate` and others. Try modifying these values in order to raise the best accuracy score. Can you raise it to 0.769 or higher?

<b>Hint:</b> Focus on a single hyperparameter and try different values higher and lower than the current value that yields the best accuracy. Once you are satisfied, move on to the next hyperparameter value and tune it similarly. 


In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#b31b1b>▶ </font>See <b>solution</b>.</summary>
<pre>
ab2 = AdaBoostClassifier(n_estimators=200, learning_rate=1.15).fit(tX, tY)
print(f'Accuracy: {ab2.score(vX, vY):.3f}')
</pre>
</details> 
</font>
<hr>